## Проект по SQL

### Описание
#### Проанализировать базу данных -  информация о книгах, издательствах, авторах, а также пользовательские обзоры книг и ответить на поставленные задачи

### Содержание
 - [Изучение таблиц](#step1)
 - [Задачи](#step2)
     - [Cколько книг вышло после 1 января 2000 года](#step3)
     - [Количество обзоров для кажой книги и средняя оценка](#step4)
     - [Определить издательство, которое выпустило наибольшее число книг толще 50 страниц](#step5) 
     - [Определить автора с самой высокой средней оценкой книг с учетом, что книги с 50 и более оценками](#step6)
     - [Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок](#step7)

In [1]:
import pandas as pd
from sqlalchemy import create_engine
pd.options.display.float_format = '{:,.1f}'.format

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя            
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}, echo=False)

## Изучение таблиц<a id="step1"></a>

### Таблица BOOKS

In [3]:
query = '''
SELECT *
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица AUTHORS

In [4]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица PUBLISHER


In [5]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Таблица RATINGS


In [6]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица REVIEWS


In [7]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задачи<a id="step2"></a>

### Cколько книг вышло после 1 января 2000 года<a id="step3"></a>

In [8]:
query = '''
SELECT COUNT(title) AS count_books
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

,count_books
0,819


#### С 1 января 2019 вышло 819 книг

### Количество обзоров для кажой книги и средняя оценка<a id="step4"></a>

In [9]:
query = '''
SELECT books.title, COUNT(DISTINCT reviews.review_id) AS count_reviews, AVG(ratings.rating) AS avg_rating
FROM books
LEFT JOIN reviews ON books.book_id = reviews.book_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
            
GROUP BY books.title
LIMIT 10;
'''
mean = pd.io.sql.read_sql(query, con = engine)
mean

,title,count_reviews,avg_rating
0,'Salem's Lot,2,3.7
1,1 000 Places to See Before You Die,1,2.5
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.7
3,1491: New Revelations of the Americas Before C...,2,4.5
4,1776,4,4.0
5,1st to Die (Women's Murder Club #1),4,3.5
6,2nd Chance (Women's Murder Club #2),3,3.0
7,4th of July (Women's Murder Club #4),3,4.0
8,A Beautiful Mind,2,4.2
9,A Bend in the Road,3,3.2


### Определить издательство, которое выпустило наибольшее число книг толще 50 страниц <a id="step5"></a>

In [10]:
query = '''
SELECT publishers.publisher, COUNT(books.book_id) AS count_books
FROM books
LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.book_id IN
    (SELECT books.book_id
     FROM books
     GROUP BY books.book_id
     HAVING books.num_pages > 50)
GROUP BY publishers.publisher
ORDER BY COUNT(books.book_id) DESC
LIMIT 1;

'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_books
0,Penguin Books,42


#### Таким является издательство Penguin Books

### Определить автора с самой высокой средней оценкой книг с учетом, что книги с 50 и более оценками <a id="step6"></a>

In [13]:
query = '''
SELECT authors.author, AVG(ratings.rating)
FROM books
LEFT JOIN authors ON books.author_id = authors.author_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
WHERE books.book_id IN

    (SELECT books.book_id
     FROM books
     LEFT JOIN ratings ON books.book_id = ratings.book_id
            
GROUP BY books.book_id
HAVING count(ratings.rating) > 49)

GROUP BY authors.author
ORDER BY AVG(ratings.rating) DESC
LIMIT 10;

'''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.3
1,Markus Zusak/Cao Xuân Việt Khương,4.3
2,J.R.R. Tolkien,4.2
3,Louisa May Alcott,4.2
4,Rick Riordan,4.1
5,William Golding,3.9
6,J.D. Salinger,3.8
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.8
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.8
9,Lois Lowry,3.8


#### Автор с самой высокой среденей оценкой с учетом условия - J.K. Rowling, а ее оценка 4,3

### Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.<a id="step7"></a>

In [12]:
query = '''
SELECT AVG(sub.count)
FROM 
    (SELECT count(*)
     FROM reviews
     WHERE username IN
         (SELECT username
          FROM ratings
          GROUP BY username
          HAVING count(rating) > 50)
    GROUP BY username) AS sub;

'''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.3


#### 24 - среднее количество обзоров от пользователей, которые поставили больше 50 оценок